In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from scipy.sparse import csr_matrix
from mlxtend.frequent_patterns import apriori, association_rules

reduce_memory: Sử dụng để giảm bộ nhớ sử dụng bằng cách chuyển đổi kiểu dữ liệu của các cột trong DataFrame. Nó kiểm tra từng cột trong DataFrame và nếu kiểu dữ liệu của cột đó là 'float64', nó chuyển đổi nó sang 'float16' để tiết kiệm bộ nhớ. Tương tự, nếu kiểu dữ liệu của cột là 'int64', nó chuyển đổi nó sang 'int16'. Điều này giúp giảm bộ nhớ sử dụng của DataFrame khi xử lý dữ liệu lớn


In [3]:
# Load data
#games_df =pd.read_csv('steam_games.csv')
#users_df = pd.read_csv('steam_users.csv')
#metadata_df = pd.read_csv('games_metadata.csv')
#recommendations_df = reduce_memory(pd.read_csv('recommendations.csv'))
games_df =pd.read_csv('processed_games_for_content-based.csv')
games_df.head()

,name,developer,publisher,popular_tags,game_details,genre,ID,genre_publisher_developer,genre_popular_tags_developer,genre_popular_tags_game_details,genre_publisher_developer_game_details
0,DayZ,BohemiaInteractive,"BohemiaInteractive,BohemiaInteractive","Survival,Zombies,OpenWorld,Multiplayer,PvP,Mas...","Multi-player,OnlineMulti-Player,SteamWorkshop,...","Action,Adventure,MassivelyMultiplayer",dayz,"Action,Adventure,MassivelyMultiplayerBohemiaIn...","Action,Adventure,MassivelyMultiplayerSurvival,...","Action,Adventure,MassivelyMultiplayerSurvival,...","Action,Adventure,MassivelyMultiplayerBohemiaIn..."
1,EVE Online,CCP,"CCP,CCP","Space,MassivelyMultiplayer,Sci-fi,Sandbox,MMOR...","Multi-player,OnlineMulti-Player,MMO,Co-op,Onli...","Action,FreetoPlay,MassivelyMultiplayer,RPG,Str...",eveonline,"Action,FreetoPlay,MassivelyMultiplayer,RPG,Str...","Action,FreetoPlay,MassivelyMultiplayer,RPG,Str...","Action,FreetoPlay,MassivelyMultiplayer,RPG,Str...","Action,FreetoPlay,MassivelyMultiplayer,RPG,Str..."
2,TERA,"Bluehole,Inc.","EnMasseEntertainment,EnMasseEntertainment","FreetoPlay,MMORPG,MassivelyMultiplayer,RPG,Ope...","Multi-player,MMO,Co-op,SteamTradingCards,Parti...","Action,Adventure,FreetoPlay,MassivelyMultiplay...",tera,"Action,Adventure,FreetoPlay,MassivelyMultiplay...","Action,Adventure,FreetoPlay,MassivelyMultiplay...","Action,Adventure,FreetoPlay,MassivelyMultiplay...","Action,Adventure,FreetoPlay,MassivelyMultiplay..."
3,Stonehearth,RadiantEntertainment,"(none),(none)","CityBuilder,Building,Sandbox,Strategy,Survival...","Single-player,Multi-player,OnlineMulti-Player,...","Indie,Simulation,Strategy",stonehearth,"Indie,Simulation,Strategy(none),(none)RadiantE...","Indie,Simulation,StrategyCityBuilder,Building,...","Indie,Simulation,StrategyCityBuilder,Building,...","Indie,Simulation,Strategy(none),(none)RadiantE..."
4,Call of Duty®: Black Ops,Treyarch,"Activision,Activision","Action,FPS,Zombies,Multiplayer,Shooter,Singlep...","Single-player,Multi-player,Co-op,SteamAchievem...",Action,callofdutyblackops,"ActionActivision,ActivisionTreyarch","ActionAction,FPS,Zombies,Multiplayer,Shooter,S...","ActionAction,FPS,Zombies,Multiplayer,Shooter,S...","ActionActivision,ActivisionTreyarchSingle-play..."


Thực hiện một phép gộp (merge) giữa hai DataFrame games_df và metadata_df dựa trên cột 'app_id'. Kết quả là một DataFrame mới combined_data chứa thông tin từ cả hai DataFrame dựa trên trường 'app_id'.
Lấy một mẫu ngẫu nhiên từ combined_data gồm 10,000 hàng (dòng) cho ra một DataFrame mới sample_data.
Loại bỏ các hàng có giá trị NaN của sample_data.
In ra 5 hàng đầu tiên của DataFrame df_cleaned 

In [7]:
#combined_data = pd.merge(games_df, metadata_df, on='app_id')

df_cleaned = games_df.dropna()
df_cleaned.head()

,name,developer,publisher,popular_tags,game_details,genre,ID,genre_publisher_developer,genre_popular_tags_developer,genre_popular_tags_game_details,genre_publisher_developer_game_details
0,DayZ,BohemiaInteractive,"BohemiaInteractive,BohemiaInteractive","Survival,Zombies,OpenWorld,Multiplayer,PvP,Mas...","Multi-player,OnlineMulti-Player,SteamWorkshop,...","Action,Adventure,MassivelyMultiplayer",dayz,"Action,Adventure,MassivelyMultiplayerBohemiaIn...","Action,Adventure,MassivelyMultiplayerSurvival,...","Action,Adventure,MassivelyMultiplayerSurvival,...","Action,Adventure,MassivelyMultiplayerBohemiaIn..."
1,EVE Online,CCP,"CCP,CCP","Space,MassivelyMultiplayer,Sci-fi,Sandbox,MMOR...","Multi-player,OnlineMulti-Player,MMO,Co-op,Onli...","Action,FreetoPlay,MassivelyMultiplayer,RPG,Str...",eveonline,"Action,FreetoPlay,MassivelyMultiplayer,RPG,Str...","Action,FreetoPlay,MassivelyMultiplayer,RPG,Str...","Action,FreetoPlay,MassivelyMultiplayer,RPG,Str...","Action,FreetoPlay,MassivelyMultiplayer,RPG,Str..."
2,TERA,"Bluehole,Inc.","EnMasseEntertainment,EnMasseEntertainment","FreetoPlay,MMORPG,MassivelyMultiplayer,RPG,Ope...","Multi-player,MMO,Co-op,SteamTradingCards,Parti...","Action,Adventure,FreetoPlay,MassivelyMultiplay...",tera,"Action,Adventure,FreetoPlay,MassivelyMultiplay...","Action,Adventure,FreetoPlay,MassivelyMultiplay...","Action,Adventure,FreetoPlay,MassivelyMultiplay...","Action,Adventure,FreetoPlay,MassivelyMultiplay..."
3,Stonehearth,RadiantEntertainment,"(none),(none)","CityBuilder,Building,Sandbox,Strategy,Survival...","Single-player,Multi-player,OnlineMulti-Player,...","Indie,Simulation,Strategy",stonehearth,"Indie,Simulation,Strategy(none),(none)RadiantE...","Indie,Simulation,StrategyCityBuilder,Building,...","Indie,Simulation,StrategyCityBuilder,Building,...","Indie,Simulation,Strategy(none),(none)RadiantE..."
4,Call of Duty®: Black Ops,Treyarch,"Activision,Activision","Action,FPS,Zombies,Multiplayer,Shooter,Singlep...","Single-player,Multi-player,Co-op,SteamAchievem...",Action,callofdutyblackops,"ActionActivision,ActivisionTreyarch","ActionAction,FPS,Zombies,Multiplayer,Shooter,S...","ActionAction,FPS,Zombies,Multiplayer,Shooter,S...","ActionActivision,ActivisionTreyarchSingle-play..."


In [9]:
# Tạo một vectorizer TF-IDF cho các đặc trưng nội dung
tfidf_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')

# Tạo ma trận TF-IDF từ các đặc trưng nội dung (ví dụ: name, popular_tags, game_details, genre)
tfidf_matrix = tfidf_vectorizer.fit_transform(df_cleaned['name'] + ' ' + df_cleaned['popular_tags'] + ' ' + df_cleaned['game_details'] + ' ' + df_cleaned['genre'])

# Tính độ tương tự cosine giữa các trò chơi
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

# Lấy top 5 gợi ý cho một trò chơi cụ thể (ví dụ: game ở vị trí thứ 1)
game_index = 0
similar_indices = cosine_similarities[game_index].argsort()[:-6:-1]

# Lấy danh sách các trò chơi được gợi ý
recommended_games = [df_cleaned['name'].iloc[i] for i in similar_indices]

print(f'Recommended games for the first game are: {recommended_games}')

#In ra các dặc trưng của các game từ danh sách đã được gợi ý
recommended_games_test = df_cleaned.iloc[similar_indices]

for index, row in recommended_games_test.iterrows():
    print("Name:", row['name'])
    print("popular_tags:", row['popular_tags'])
    print("genre:", row['genre'])
    print("------------")


Recommended games for the first game are: ['DayZ', 'Miscreated', 'Badland Bandits', 'Unturned', 'Squad']
Name: DayZ
popular_tags: Survival,Zombies,OpenWorld,Multiplayer,PvP,MassivelyMultiplayer,Action,EarlyAccess,Simulation,FPS,Post-apocalyptic,SurvivalHorror,Shooter,Sandbox,Adventure,Indie,Co-op,Atmospheric,Horror,Military
genre: Action,Adventure,MassivelyMultiplayer
------------
Name: Miscreated
popular_tags: Survival,OpenWorld,Multiplayer,Zombies,Post-apocalyptic,Crafting,PvP,MassivelyMultiplayer,Action,Sandbox,Co-op,EarlyAccess,FPS,First-Person,Horror,Simulation,WalkingSimulator,Indie,Singleplayer,MMORPG
genre: Action,Adventure,Indie,MassivelyMultiplayer,RPG,Simulation,Strategy
------------
Name: Badland Bandits
popular_tags: EarlyAccess,Racing,MassivelyMultiplayer,Action,EarlyAccess,Simulation,RPG,FreetoPlay,Steampunk,Multiplayer
genre: Action,MassivelyMultiplayer,Racing,RPG,Simulation,EarlyAccess
------------
Name: Unturned
popular_tags: FreetoPlay,Survival,Zombies,Multiplayer,Op

In [6]:
# Content-Based Filtering dựa trên title
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(games_df['title'])
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

# Lấy top 5 gợi ý cho một trò chơi cụ thể (ví dụ: trò chơi đầu tiên)
similar_indices = cosine_similarities[0].argsort()[::-1][:5]
recommended_games = [games_df['title'].iloc[i] for i in similar_indices]
print(f'Recommended games for the first game are: {recommended_games}')

Recommended games for the first game are: ['Jigsaw 360', 'Girl Jigsaw 2', 'Red vs Blue 360', 'Jigsaw Puzzles Infinite', 'Jigsaw Pieces 3 - Fantasy']
